In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
caseid = test_data['case_id']
caseid = pd.Series(caseid).array

In [4]:
train_x = train_data.drop(['Stay'], axis=1)
train_y = train_data['Stay']

In [5]:
temp = pd.concat([train_x,test_data],keys=[0,1])

In [6]:
df1=pd.read_csv('train_data_dict.csv') 
df1.head()

,Column,Description
0,case_id,Case_ID registered in Hospital
1,Hospital_code,Unique code for the Hospital
2,Hospital_type_code,Unique code for the type of Hospital
3,City_Code_Hospital,City Code of the Hospital
4,Hospital_region_code,Region Code of the Hospital


In [7]:
temp.isna().sum()

case_id                                 0
Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                             148
patientid                               0
City_Code_Patient                    6689
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
dtype: int64

In [8]:
temp['Bed Grade'].fillna(temp['Bed Grade'].mode()[0],inplace=True)
temp['City_Code_Patient'].fillna(temp['City_Code_Patient'].mode()[0], inplace = True)

In [9]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 455495 entries, (0, 0) to (1, 137056)
Data columns (total 17 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            455495 non-null  int64  
 1   Hospital_code                      455495 non-null  int64  
 2   Hospital_type_code                 455495 non-null  object 
 3   City_Code_Hospital                 455495 non-null  int64  
 4   Hospital_region_code               455495 non-null  object 
 5   Available Extra Rooms in Hospital  455495 non-null  int64  
 6   Department                         455495 non-null  object 
 7   Ward_Type                          455495 non-null  object 
 8   Ward_Facility_Code                 455495 non-null  object 
 9   Bed Grade                          455495 non-null  float64
 10  patientid                          455495 non-null  int64  
 11  City_Code_Patient            

In [10]:
temp1=temp.drop(['case_id','patientid'],axis=1)

In [11]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()

In [12]:
obj = temp1.select_dtypes(include ='object')
obj_col = obj.columns.tolist()
for i in range(0,obj.shape[1]):
    label.fit(temp1[obj_col[i]])
    temp1[obj_col[i]] = label.transform(temp1[obj_col[i]])

In [13]:
temp1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 455495 entries, (0, 0) to (1, 137056)
Data columns (total 15 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Hospital_code                      455495 non-null  int64  
 1   Hospital_type_code                 455495 non-null  int32  
 2   City_Code_Hospital                 455495 non-null  int64  
 3   Hospital_region_code               455495 non-null  int32  
 4   Available Extra Rooms in Hospital  455495 non-null  int64  
 5   Department                         455495 non-null  int32  
 6   Ward_Type                          455495 non-null  int32  
 7   Ward_Facility_Code                 455495 non-null  int32  
 8   Bed Grade                          455495 non-null  float64
 9   City_Code_Patient                  455495 non-null  float64
 10  Type of Admission                  455495 non-null  int32  
 11  Severity of Illness          

In [14]:
x_train=temp1.xs(0)
x_train.shape

(318438, 15)

In [15]:
x_test = temp1.xs(1)
x_test.shape

(137057, 15)

In [16]:
y_train = train_y

In [17]:
from lightgbm import LGBMClassifier
lgb=LGBMClassifier()

In [18]:
lgb.fit(x_train,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [19]:
#knn.score(x_train,y_train)

In [20]:
y_pred=lgb.predict(x_test)

In [21]:
df=pd.DataFrame({'case_id':caseid,'Stay':y_pred})
df

,case_id,Stay
0,318439,0-10
1,318440,51-60
2,318441,21-30
3,318442,21-30
4,318443,51-60
...,...,...
137052,455491,21-30
137053,455492,0-10
137054,455493,11-20
137055,455494,11-20


In [22]:
df.to_csv (r'C:\Users\V ABHIGNA\Desktop\lgb-drop.csv', index = False, header=True)